## 2022/11/08 針對 Dofloo 單一個樣本，4 種 ASG 的結果進行分析

In [65]:
import pandas as pd
import pickle
import sys
from collections import Counter

從上層資料夾import模組，但build()的相對路徑會出錯，還是直接把模組搬過來測。
- https://www.geeksforgeeks.org/python-import-from-parent-directory/
- https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
- https://stackoverflow.com/questions/39125532/file-does-not-exist-in-jupyter-notebook

In [2]:
 # setting path to import cc_module
# import sys
# import os
# current = os.path.abspath('')
# # current = os.path.dirname(os.path.realpath(__file__))
# parent = os.path.dirname(current)
# sys.path.append(parent)
from cc_module import build
from cc_module import AttackGraph, Node, Edge, FileTable

In [3]:
asg, g = build('Dofloo', '9a37fcc7eab08d59532bc5c66390bc30.bin')

path: ../trace/Dofloo/9a37fcc7eab08d59532bc5c66390bc30.bin
Nodes after reduction:  36


In [10]:
node_set = [node.name for node in asg.graph.keys()]
print('# of nodes:',len(node_set))
print('# of edges:',len(asg.edges),'(edges equal to steps)')

# of nodes: 36
# of edges: 40 (edges equal to steps)


In [31]:
nodeType_counter = Counter({n:0 for n in ['f','c','n','p','m_addr','pipe','else']})
for node in asg.graph.keys():
    nodeType_counter[node.type] += 1
nodeType_counter

Counter({'f': 7, 'c': 1, 'n': 11, 'p': 17, 'm_addr': 0, 'pipe': 0, 'else': 0})

### Calc frequency of node appearance
指出指入都計算一次，暫無分開統計指出幾次，指入幾次。

In [33]:
# calc node's frequency
node_counter = Counter({n:0 for n in node_set})
edge_counter = Counter()
for (nodes, edge) in asg.edges.items():
    # print(nodes, edge)
    # print(edge.name)
    src_node, dst_node, _ = nodes # 其實底線_等同於 edge.name
    # print(src_node.name, dst_node.name)
    node_counter[src_node.name] += 1
    node_counter[dst_node.name] += 1
    edge_counter[edge.name] += 1
    # break
node_counter

Counter({'malware': 8,
         'uname': 1,
         'sh': 12,
         '1529': 16,
         'sed': 14,
         '/etc/rc.local': 6,
         '/etc/rc.d/rc.local': 1,
         '/etc/init.d/boot.local': 1,
         '1530': 4,
         '1531': 1,
         '1532': 1,
         '/dev/urandom': 1,
         'eth0': 1,
         'eth1': 1,
         'eth2': 1,
         'eth3': 1,
         'eth4': 1,
         'eth5': 1,
         'eth6': 1,
         'eth7': 1,
         'eth8': 1,
         'eth9': 1,
         '183.131.83.38:50050': 1,
         '/proc/net/dev': 1,
         '/sys/devices/system/cpu/online': 1,
         '/proc/stat': 1})

In [34]:
edge_counter

Counter({'read': 18,
         'clone': 10,
         'exec': 6,
         'write': 3,
         'open': 2,
         'connect': 1})

### 包裝成 function，印出統計結果
先印在notebook output裡，太長再改成輸出到txt中。

In [66]:
def analysis_graph(asg: AttackGraph):
    node_set = [node.name for node in asg.graph.keys()]
    print('# of nodes:',len(node_set))
    print('# of edges:',len(asg.edges),'(edges equal to steps)\n')

    nodeType_counter = Counter({n:0 for n in ['f','c','n','p','m_addr','pipe','else']})
    for node in asg.graph.keys():
        nodeType_counter[node.type] += 1
    print('# of node type:\n',nodeType_counter)

    node_counter = Counter({n:0 for n in node_set})
    edge_counter = Counter()
    for (nodes, edge) in asg.edges.items():
        # print(nodes, edge)
        # print(edge.name)
        src_node, dst_node, _ = nodes # 其實底線_等同於 edge.name
        # print(src_node.name, dst_node.name)
        node_counter[src_node.name] += 1
        node_counter[dst_node.name] += 1
        edge_counter[edge.name] += 1
    print('\nfrequency of node:\n',node_counter)
    print('\nfrequency of edge:\n',edge_counter)
    print('\n# of syscall',len(edge_counter))

- 這邊直接寫入檔案 V1.txt, V2.txt ...
- https://stackoverflow.com/questions/7152762/how-to-redirect-print-output-to-a-file

In [68]:
# filename = 'Dofloo_V1_ALL有畫出-Memory-and-System.pkl'
# with open(filename, 'rb') as inp:
#     asg_all = pickle.load(inp)

filename_lst = ['Dofloo_V1_ALL_Memory_and_System.pkl','Dofloo_V2_ALL_Memory_sink.pkl',
    'Dofloo_V3_ALL_wo_system.pkl','Dofloo_V4_ALL_wo_system_memory.pkl']
    
for i,filename in enumerate(filename_lst):
    orig_stdout = sys.stdout
    f = open(f'Dofloo_V{i+1}.txt', 'w')
    sys.stdout = f

    with open(filename, 'rb') as inp:
        asg_object = pickle.load(inp)
        analysis_graph(asg_object)

    sys.stdout = orig_stdout
    f.close()

In [64]:
analysis_graph(asg_all)

# of nodes: 440
# of edges: 803 (edges equal to steps)
# of node type: Counter({'m_addr': 181, 'else': 170, 'f': 60, 'p': 17, 'n': 11, 'c': 1, 'pipe': 0})
frequency of node: Counter({'sed': 407, 'sh': 198, '1527': 164, '/lib/x86_64-linux-gnu/libc.so.6': 36, '1526': 27, 'malware': 26, '/etc/ld.so.cache': 24, '0': 24, '/lib/x86_64-linux-gnu/libselinux.so.1': 18, '/lib/x86_64-linux-gnu/libpcre.so.3': 18, '/lib/x86_64-linux-gnu/libdl.so.2': 18, '/lib/x86_64-linux-gnu/libpthread.so.0': 18, '/etc/ld.so.nohwcap': 12, '/etc/ld.so.preload': 12, '/proc/filesystems': 12, '/usr/lib/locale/locale-archive': 12, '/usr/lib/x86_64-linux-gnu/gconv/gconv-modules.cache': 12, '/etc/rc.local': 9, '8192*1024 bytes': 7, 'status:0': 7, '1513': 6, '/prober': 6, '.': 6, '/usr/local/sbin/sed': 6, '/usr/local/bin/sed': 6, '/usr/sbin/sed': 6, '/usr/bin/sed': 6, '/sbin/sed': 6, '/bin/sed': 6, '/sys/fs/selinux': 6, '/selinux': 6, '/etc/selinux/config': 6, '/usr/lib/x86_64-linux-gnu/charset.alias': 6, '/usr/share/loca

In [62]:
# 志剛發現 asg data structure 缺少了一些 syscall，如下 
dict_from_net = {'set_thread_area': 1, 'ugetrlimit': 1, 'readlink': 1, 'getcwd': 1, 'setsid': 1, 'uname': 2, 'fcntl': 3, 'fchown': 3, 'fchmod': 3, 'rename': 3, 'waitpid': 6, 'getuid': 6, 'getgid': 6, 'getppid': 6, 'getegid': 6, 'wait4': 6, 'rt_sigreturn': 6, 'prlimit64': 6, 'umask': 6, 'set_tid_address': 7, 'futex': 8, 'getpid': 9, 'set_robust_list': 11, 'arch_prctl': 12, 'geteuid': 12, 'statfs': 12, 'execve': 13, 'exit_group': 13, 'clone': 16, 'write': 28, 'connect': 39, '_newselect': 39, 'getsockopt': 39, 'send': 39, 'brk': 40, 'socket': 40, 'stat': 48, 'access': 66, 'fstat': 78, 'ioctl': 91, 'mprotect': 100, 'mmap': 138, 'openat': 143, 'time': 189, 'fstat64': 299, 'mmap2': 303, 'munmap': 312, 'nanosleep': 336, 'rt_sigaction': 385, 'rt_sigprocmask': 624, 'lseek': 1500, 'open': 2041, 'read': 2101, 'close': 2463}
dict_from_asg = {'time': 150, 'mprotect': 100, 'fstat': 75, 'open': 68, 'read': 63, 'mmap': 60, 'stat': 48, 'access': 30, 'brk': 28, 'munmap': 16, 'exit_group': 13, 'arch_prctl': 12, 'statfs': 12, 'set_robust_list': 11, 'clone': 10, 'getpid': 9, 'set_tid_address': 7, 'futex': 7, 'waitpid': 6, 'getuid': 6, 'getgid': 6, 'geteuid': 6, 'getppid': 6, 'getegid': 6, 'exec': 6, 'prlimit64': 6, 'umask': 6, 'write': 6, 'fcntl': 3, 'fchown': 3, 'fchmod': 3, 'fstat64': 2, 'mmap2': 2, 'nanosleep': 2, 'set_thread_area': 1, 'ugetrlimit': 1, 'readlink': 1, 'getcwd': 1, 'setsid': 1, '_newselect': 1, 'getsockopt': 1, 'lseek': 1}
setA = set(dict_from_net.keys())
setB = set(dict_from_asg.keys())
setA - setB

{'close',
 'connect',
 'execve',
 'ioctl',
 'openat',
 'rename',
 'rt_sigaction',
 'rt_sigprocmask',
 'rt_sigreturn',
 'send',
 'socket',
 'uname',
 'wait4'}

In [59]:
'exec' in setB

True